In [11]:
!pip install python-dotenv



In [2]:
import pandas as pd
import sys  
import os
import nest_asyncio
import numpy as np
from telethon.sync import TelegramClient
import re
import emoji
import csv
from dotenv import load_dotenv


# Selecting 5 channels nand Scrape messages 

In [ ]:


# Allow nested asyncio loops (needed in notebooks like Jupyter)
nest_asyncio.apply()

# Load environment variables from .env file
load_dotenv('.env')

# Retrieve API credentials
API_ID = os.getenv('TG_API_ID')
API_HASH = os.getenv('TG_API_HASH')

if API_ID is None:
    raise ValueError("API_ID not found in environment variables")
else:
    API_ID = int(API_ID)

if API_HASH is None:
    raise ValueError("API_HASH not found in environment variables")

# Async function to fetch messages and media from a channel
async def fetch_channel_messages(client, channel_handle, csv_writer, download_folder):
    channel_entity = await client.get_entity(channel_handle)
    channel_name = channel_entity.title
    async for msg in client.iter_messages(channel_entity, limit=10000):
        media_file_path = None
        if msg.media and hasattr(msg.media, 'photo'):
            filename = f"{channel_handle}_{msg.id}.jpg"
            media_file_path = os.path.join(download_folder, filename)
            await client.download_media(msg.media, media_file_path)

        csv_writer.writerow([
            channel_name,
            channel_handle,
            msg.id,
            msg.message,
            msg.date,
            media_file_path
        ])

# Main async function to run the scraper
async def run_scraper():
    client = TelegramClient('session_scraper', API_ID, API_HASH)
    await client.start()

    # Define where to save data and images
    base_folder = r"C:\Users\HP\10 Acadamy PRojects\New folder (4)\amharic-ecommerce-scraper\data"
    media_folder = os.path.join(base_folder, 'downloaded_images')
    os.makedirs(media_folder, exist_ok=True)

    csv_path = os.path.join(base_folder, 'output_telegram_data.csv')

    with open(csv_path, mode='w', newline='', encoding='utf-8') as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(['Channel Title', 'Channel Handle', 'Message ID', 'Content', 'Timestamp', 'Media File'])

  target_channels = [
            'ZemenExpress',
            'meneshayeofficial',
            'ethio_brand_collection',
            'Shewabrand',
            'qnashcom'
        ]

        for channel in target_channels:
            print(f"Scraping channel: {channel} ...")
            await fetch_channel_messages(client, channel, writer, media_folder)
            print(f"✅ Completed scraping {channel}")

    await client.disconnect()

# Run the scraper in a notebook or async environment
await run_scraper()


Scraping channel: ZemenExpress ...
✅ Completed scraping ZemenExpress
Scraping channel: meneshayeofficial ...
✅ Completed scraping meneshayeofficial
Scraping channel: ethio_brand_collection ...
✅ Completed scraping ethio_brand_collection
Scraping channel: Shewabrand ...
✅ Completed scraping Shewabrand
Scraping channel: qnashcom ...
✅ Completed scraping qnashcom


# Cleaning The Dataset

In [7]:

# Paths
base_dir = r"C:\Users\HP\10 Acadamy PRojects\New folder (4)\amharic-ecommerce-scraper\data"
input_file = os.path.join(base_dir, "output_telegram_data.csv")
output_file = os.path.join(base_dir, "output_telegram_data_cleaned.csv")

# Load data
df = pd.read_csv(input_file, encoding='utf-8')


# Clean Amharic text
def clean_amharic_text(text):
    if pd.isnull(text):
        return ""
    
    text = emoji.replace_emoji(text, replace='')  # Remove emojis
    text = re.sub(r"http\S+|www\S+|https\S+", '', text)  # URLs
    text = re.sub(r"[A-Za-z]", '', text)  # English
    text = re.sub(r"[^\u1200-\u137F\u1380-\u139F\u2D80-\u2DDF0-9፡።፣፤፥፦፧፨\s]", '', text)
    text = re.sub(r"\s+", " ", text)  # Normalize spaces
    return text.strip()

# Tokenizer
def tokenize_text(text):
    return text.split() if text else []

# Apply
df['clean_text'] = df['Content'].apply(clean_amharic_text)
df['tokens'] = df['clean_text'].apply(tokenize_text)

# Save result
df.to_csv(output_file, index=False, encoding='utf-8-sig')
print(f"✅ Cleaned data saved to:\n{output_file}")


✅ Cleaned data saved to:
C:\Users\HP\10 Acadamy PRojects\New folder (4)\amharic-ecommerce-scraper\data\output_telegram_data_cleaned.csv


In [21]:
df = pd.read_csv("C:/Users/HP/10 Acadamy PRojects/New folder (4)/amharic-ecommerce-scraper/data/output_telegram_data_cleaned.csv")
df.head()

,Channel Title,Channel Handle,Message ID,Content,Timestamp,Media File,clean_text,tokens
0,Zemen Express®,ZemenExpress,6982,💥💥...................................💥💥\n\n📌Im...,2025-06-18 06:01:10+00:00,NaN,በኤሌክትሪክየሚሰራ ለቤት መልካም መዓዛን የሚሰጥ ዋጋ፦ 1400 ብር ውስን...,"['በኤሌክትሪክየሚሰራ', 'ለቤት', 'መልካም', 'መዓዛን', 'የሚሰጥ',..."
1,Zemen Express®,ZemenExpress,6981,💥💥...................................💥💥\n\n📌 B...,2025-06-16 12:21:00+00:00,NaN,በፈለጉት አቅጣጫ ልጅዎን በምቾት ማዘል ያስችልዎታል ዋጋ፦ 2400 ብር ው...,"['በፈለጉት', 'አቅጣጫ', 'ልጅዎን', 'በምቾት', 'ማዘል', 'ያስችል..."
2,Zemen Express®,ZemenExpress,6980,NaN,2025-06-16 05:11:57+00:00,C:\Users\HP\10 Acadamy PRojects\New folder (4)...,NaN,[]
3,Zemen Express®,ZemenExpress,6979,NaN,2025-06-16 05:11:57+00:00,C:\Users\HP\10 Acadamy PRojects\New folder (4)...,NaN,[]
4,Zemen Express®,ZemenExpress,6978,NaN,2025-06-16 05:11:57+00:00,C:\Users\HP\10 Acadamy PRojects\New folder (4)...,NaN,[]


# Selecting 50 Massages from the DF

In [13]:
import pandas as pd
import os

# Load cleaned data
base_dir = r"C:\Users\HP\10 Acadamy PRojects\New folder (4)\amharic-ecommerce-scraper\data"
input_file = os.path.join(base_dir, "output_telegram_data_cleaned.csv")
df = pd.read_csv(input_file)

# Define keyword lists
price_keywords = ['ብር', 'ዋጋ', 'በ']
product_keywords = ['ልጅ', 'ሻርተ', 'እቃ', 'ቦትል', 'ኩባያ']
location_keywords = ['አዲስ', 'ቦሌ', 'ሀዋሳ', 'መጫኛ', 'ትኩሳት']

# Combine them
all_keywords = price_keywords + product_keywords + location_keywords

# Function to check if any keyword exists in text
def contains_keywords(text, keywords):
    return any(kw in str(text) for kw in keywords)

# Filter rows with relevant content
filtered_df = df[df['clean_text'].apply(lambda x: contains_keywords(x, all_keywords))]

# Limit to 50 messages
filtered_sample = filtered_df.sample(n=50, random_state=42)

# Save
output_sample_path = os.path.join(base_dir, "ner_labeling_sample.csv")
filtered_sample.to_csv(output_sample_path, index=False, encoding='utf-8-sig')

print(f"✅ Filtered 50 messages and saved to:\n{output_sample_path}")


✅ Filtered 50 messages and saved to:
C:\Users\HP\10 Acadamy PRojects\New folder (4)\amharic-ecommerce-scraper\data\ner_labeling_sample.csv


In [4]:
df = pd.read_csv("C:/Users/HP/10 Acadamy PRojects/New folder (4)/amharic-ecommerce-scraper/data/ner_labeling_sample.csv")
df.head(50
        )

,Channel Title,Channel Handle,Message ID,Content,Timestamp,Media File,clean_text,tokens
0,Zemen Express®,ZemenExpress,4150,💥💥...................................💥💥\n\n📌Fl...,2024-03-23 09:34:00+00:00,NaN,የራሱ ሪሞት ያለው ከስቲም በተጨማሪ ውብ ብርሀን የሚሰጥ 200 ውሀ የመያ...,"['የራሱ', 'ሪሞት', 'ያለው', 'ከስቲም', 'በተጨማሪ', 'ውብ', '..."
1,qnash.com - ቅናሽ ®️,qnashcom,1686,🤩 ታላቅ ቅናሽ 💯 🤩 \n❇️ Universal water-saving dish...,2022-05-31 05:56:50+00:00,NaN,ታላቅ ቅናሽ 360 100 ዋጋ፦ 350 ብር ውስን ፍሬ ነው የቀረው ጥራት ...,"['ታላቅ', 'ቅናሽ', '360', '100', 'ዋጋ፦', '350', 'ብር..."
2,Zemen Express®,ZemenExpress,999,🎯Electric Coffee Maker\n🔰300ml የሚይዝ ለ 6 ስኒ\n\n...,2022-02-03 08:22:27+00:00,C:\Users\HP\10 Acadamy PRojects\New folder (4)...,300 የሚይዝ ለ 6 ስኒ ዋጋ፦ 1450 ብር አድራሻ መገናኛ ደራርቱ ህንፃ...,"['300', 'የሚይዝ', 'ለ', '6', 'ስኒ', 'ዋጋ፦', '1450',..."
3,Shewa Brand,Shewabrand,3710,jordan 1 original 💯 \nSize 40#41#42#43\nMADE I...,2025-06-20 06:15:40+00:00,C:\Users\HP\10 Acadamy PRojects\New folder (4)...,1 40414243 አድራሻ ድሬዳዋ አሸዋ ሚና ህንፃ 1ኛ ፎቅ ላይ እንገኛለ...,"['1', '40414243', 'አድራሻ', 'ድሬዳዋ', 'አሸዋ', 'ሚና',..."
4,qnash.com - ቅናሽ ®️,qnashcom,3282,😃 የሸቶ መቀነሻ Mini spray pour\n\nየሸቶ መቀነሻ 👉\n\n ...,2023-10-13 07:30:00+00:00,NaN,የሸቶ መቀነሻ የሸቶ መቀነሻ 200 ብር ውስን ፍሬ ነው የቀረው ጥራት ዋስ...,"['የሸቶ', 'መቀነሻ', 'የሸቶ', 'መቀነሻ', '200', 'ብር', 'ው..."
5,qnash.com - ቅናሽ ®️,qnashcom,2968,📣🔥Saachi Electric Kettle\nሳቺ ውሃ ማፍያ \n☄️ High ...,2023-05-31 09:29:04+00:00,NaN,ሳቺ ውሃ ማፍያ 18 360 2200 ዋጋ 2200 ብር ውስን ፍሬ ነው የቀረ...,"['ሳቺ', 'ውሃ', 'ማፍያ', '18', '360', '2200', 'ዋጋ',..."
6,Zemen Express®,ZemenExpress,6243,💥💥...................................💥💥\n\n📌Pa...,2025-02-18 06:01:24+00:00,C:\Users\HP\10 Acadamy PRojects\New folder (4)...,700 ዋጋ፦ 1600 ብር ውስን ፍሬ ነው ያለው አድራሻ ቁመገናኛ መሰረት ...,"['700', 'ዋጋ፦', '1600', 'ብር', 'ውስን', 'ፍሬ', 'ነው'..."
7,qnash.com - ቅናሽ ®️,qnashcom,713,❇️ Automatic Vacuum Sealer Machine የላስቲክ ማሸጊያ...,2021-11-15 17:53:07+00:00,NaN,የላስቲክ ማሸጊያ ላስቲክ ዉስጥ ያለዉን አየር ስቦ ማሸግ የሚችል ፍራፍሬ ...,"['የላስቲክ', 'ማሸጊያ', 'ላስቲክ', 'ዉስጥ', 'ያለዉን', 'አየር'..."
8,Zemen Express®,ZemenExpress,5011,💥💥...................................💥💥\n\n📌4p...,2024-08-12 18:56:06+00:00,C:\Users\HP\10 Acadamy PRojects\New folder (4)...,4 90 ዋጋ፦ 550 ብር አድራሻ ቁመገናኛ መሰረት ደፋር ሞል ሁለተኛ ፎቅ...,"['4', '90', 'ዋጋ፦', '550', 'ብር', 'አድራሻ', 'ቁመገናኛ..."
9,መነሻዬ,meneshayeofficial,625,Rc cars and transformables\n\nበሪሞት ኮንትሮል የሚንቀሳ...,2023-09-11 08:42:09+00:00,NaN,በሪሞት ኮንትሮል የሚንቀሳቀሱ መኪኖች በአይነት በአይነት ከመነሻዬ ቢፈልጉ...,"['በሪሞት', 'ኮንትሮል', 'የሚንቀሳቀሱ', 'መኪኖች', 'በአይነት', ..."


In [1]:

# # === CONFIG ===
# base_dir = r"C:\Users\HP\10 Acadamy PRojects\New folder (4)\amharic-ecommerce-scraper\data"
# input_file = os.path.join(base_dir, "50_labeling_samples.csv")
# output_conll = os.path.join(base_dir, "ner_data.conll")

# # === LOAD DATA ===
# df = pd.read_csv(input_file)

# # === LABELING FUNCTION ===
# def label_tokens_to_conll(df):
#     all_sentences = []

#     for i, row in df.iterrows():
#         print(f"\n🔢 Message {i+1}/{len(df)}")
#         tokens = eval(row['tokens']) if isinstance(row['tokens'], str) else row['tokens']
#         sentence_labels = []

#         for token in tokens:
#             print(f"\n🔤 Token: {token}")
#             tag = input("👉 Enter tag [B-Product, I-Product, B-PRICE, I-PRICE, B-LOC, I-LOC, O]: ").strip()
#             while tag not in ['B-Product', 'I-Product', 'B-PRICE', 'I-PRICE', 'B-LOC', 'I-LOC', 'O']:
#                 print("❌ Invalid tag. Try again.")
#                 tag = input("👉 Enter tag: ").strip()
#             sentence_labels.append((token, tag))

#         all_sentences.append(sentence_labels)

#     # Save in CoNLL format
#     with open(output_conll, 'w', encoding='utf-8') as f:
#         for sentence in all_sentences:
#             for token, tag in sentence:
#                 f.write(f"{token} {tag}\n")
#             f.write("\n")  # Sentence break

#     print(f"\n✅ Done! Labeled data saved to:\n{output_conll}")

# # === RUN ===
# label_tokens_to_conll(df)


In [10]:
import pandas as pd
import os

# === CONFIG ===
base_dir = r"C:\Users\HP\10 Acadamy PRojects\New folder (4)\amharic-ecommerce-scraper\data"
input_file = os.path.join(base_dir, "C:/Users/HP/10 Acadamy PRojects/New folder (4)/amharic-ecommerce-scraper/data/50_labeling_samples.csv")
output_conll = os.path.join(base_dir, "labeled_data.conll")

# === TOKENIZER ===
def tokenize_message(message):
    if isinstance(message, str):
        return message.split()
    return []

# === LABELING FUNCTION ===
def annotate_message(tokens):
    labeled_tokens = []
    print("\nStart labeling (press Enter = 'O'):")
    for token in tokens:
        tag = input(f"🔤 Token: {token}\n👉 Tag [default=O]: ").strip()
        if tag == "":
            tag = "O"
        while tag not in ['B-Product', 'I-Product', 'B-LOC', 'I-LOC', 'B-PRICE', 'I-PRICE', 'O']:
            print("❌ Invalid tag. Try again.")
            tag = input(f"👉 Tag for '{token}' [default=O]: ").strip()
            if tag == "":
                tag = "O"
        labeled_tokens.append((token, tag))
    return labeled_tokens

# === SAVE TO CoNLL ===
def save_to_conll(labeled_data, output_file):
    with open(output_file, 'w', encoding='utf-8') as f:
        for sentence in labeled_data:
            for token, label in sentence:
                f.write(f"{token} {label}\n")
            f.write("\n")
    print(f"\n✅ Saved labeled data to:\n{output_file}")

# === MAIN EXECUTION FOR NOTEBOOK ===
def run_labeling(start_row=0, num_rows=5):
    df = pd.read_csv(input_file)

    labeled_data = []

    df_subset = df.iloc[start_row : start_row + num_rows]

    for idx, row in df_subset.iterrows():
        message = row['clean_text']
        tokens = tokenize_message(message)

        if not tokens:
            continue

        print(f"\n🟩 Message {idx+1}/{len(df)}:\n➡️ {message}")
        labeled_tokens = annotate_message(tokens)
        labeled_data.append(labeled_tokens)

    save_to_conll(labeled_data, output_conll)


In [14]:
run_labeling(start_row=0, num_rows=51)



🟩 Message 1/50:
➡️ የራሱ ሪሞት ያለው ከስቲም በተጨማሪ ውብ ብርሀን የሚሰጥ 200 ውሀ የመያዝ አቅም ያለው በኤሌክትሪክ የሚሰራ ዋጋ፦ 1650ብር ውስን ፍሬ ነው ያለው አድራሻ ቁመገናኛ መሰረት ደፋር ሞል ሁለተኛ ፎቅ ቢሮ ቁ 0506 ቁፒያሳ ጊዮርጊስ አደባባይ ራመትታቦርኦዳህንፃ 1ኛ ፎቅ ሱቅ ቁ 1 107 0902660722 0928460606 ፒያሳ ቅርንጫፍ 0941337070 በ ለማዘዝ ይጠቀሙ ለተጨማሪ ማብራሪያ የቴሌግራም ገፃችን

Start labeling (press Enter = 'O'):

🟩 Message 2/50:
➡️ ታላቅ ቅናሽ 360 100 ዋጋ፦ 350 ብር ውስን ፍሬ ነው የቀረው ጥራት ዋስትና ቅናሽ ማሳሰብያ ሱቃችን ሲመጡ ትክክለኛ የኛ ሱቅ መሆኑ ያረጋግጡ እንዳንድ ሰወች 329 ነው እያሉ የማይሆን ዕቃ በማይሆን ዋጋ እየሸጡ ስለሆነ ጥንቃቄ አድርጉ የራሳችን ሎጎ መኖሩን እና 329 መሆኑ ያረጋግጡ ዘፍመሽ ግራንድ ሞል 3 ተኛ ፎቅ ከሊፍት እንደወረዱ በግራ በኩል 329 የሚል ባነር ማስታወቂያ ያያሉ ባነሩ ባለበት ዞር እንዳሉ ስልክ 0946966440 0905464599 251118639952

Start labeling (press Enter = 'O'):

🟩 Message 3/50:
➡️ 300 የሚይዝ ለ 6 ስኒ ዋጋ፦ 1450 ብር አድራሻ መገናኛ ደራርቱ ህንፃ ጎን መሰረት ደፋር ሞል ሁለተኛ ፎቅ ቢሮ ቁ 0506 ያሉበት ድረስ በነፃ እናደርሳለን 0902660722 0928460606 በ ለማዘዝ ይጠቀሙ ለተጨማሪ ማብራሪያ የቴሌግራም ገፃችን

Start labeling (press Enter = 'O'):

🟩 Message 4/50:
➡️ 1 40414243 አድራሻ ድሬዳዋ አሸዋ ሚና ህንፃ 1ኛ ፎቅ ላይ እንገኛለን የቴሌግራም ቻናላችንን ይቀላቀሉ የቤት ቁጥር 109 እና 110 በ